In [1]:
import yaml

In [2]:
# Read configuration
with open("config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

config


{'config_dummy': {'a': 2, 'b': 10, 'c': 256}}

In [3]:
# Read master
with open("master.yaml", "r") as f:
    master = yaml.load(f, Loader=yaml.FullLoader)
    
master


{'gen1': {'script': ['do_operations', 'print_output']}}

In [4]:
# Read recipes
with open("recipes.yaml", "r") as f:
    recipes = yaml.load(f, Loader=yaml.FullLoader)

recipes

{'do_operations': {'script': ['add', 'multiply', 'power']},
 'print_output': {'script': ['print_result']}}

In [5]:
for gen in sorted(master.keys()):
    print("Now browsing generation: " + gen)
    for recipe in master[gen]['script']:
        print("Now browsing recipe: " + recipe)
        print(recipes[recipe]['script'])
    print()

Now browsing generation: gen1
Now browsing recipe: do_operations
['add', 'multiply', 'power']
Now browsing recipe: print_output
['print_result']



In [6]:
# Try get function code
from src import power

print('*********')
print(power.get_str())
print('*********')
print(power.get_output_str())
print('*********')
print(power.get_output_type_hint_str())
print('*********')
print(power.get_call_str())
print('*********')
print(power.get_assignation_call_str())
print('*********')
print(power.get_signature())
print('*********')
print(power.get_parameters())
print('*********')
print(power.dic_imports)
print('*********')



*********
def power_function(b: int, c: int) -> int:
    # Returns a at the power of b
    return np.power(b, c)

*********
output_power
*********
int
*********
power_function(b, c)
*********
output_power = power_function(b, c)
*********
(b: int, c: int) -> int
*********
OrderedDict([('b', <class 'int'>), ('c', <class 'int'>)])
*********
{'numpy': 'np'}
*********


In [7]:
# Generate all blocks in a file
import src.blocks as blocks
from jinja2 import Environment

# Generate all blocks
file_path = "output_jinja_test_gen1.py"
for gen in sorted(master.keys()):
    print("Now browsing generation: " + gen)
    print('\n')
    dic_blocks = {}
    for recipe in master[gen]['script']:
        print("Now browsing recipe: " + recipe)
        print('')
        for block in recipes[recipe]['script']:
            print("Now browsing block: " + block)
            dic_blocks[block] = getattr(blocks,block)
            
            
# Build template with all blocks separated by \n
dic_blocks_str = {k: v.get_str() for k,v in dic_blocks.items()}
str_jinja = "\n".join([f"{{{{{k}}}}}" for k in dic_blocks.keys()])
template = Environment().from_string(str_jinja)
with open(file_path, "w") as f:
    f.write(template.render(**dic_blocks_str))
                

Now browsing generation: gen1


Now browsing recipe: do_operations

Now browsing block: add
Now browsing block: multiply
Now browsing block: power
Now browsing recipe: print_output

Now browsing block: print_result


In [8]:
# Try merging blocks

# Add takes a and b and returns output_add
# Power takes b and c and returns output_power
# Let's add to output c instead of output add
from typing import OrderedDict
from src import add

add.output = OrderedDict([("c", add.output["output_add"])])

add_power = add.merge_block(
    dic_blocks["power"],
    "add_power",
    docstring="Test with long docstring",
    output = dic_blocks["power"].output,
)

In [9]:
print('*********')
print(add_power.get_str())
print('*********')
print(add_power.get_output_str())
print('*********')
print(add_power.get_output_type_hint_str())
print('*********')
print(add_power.get_call_str())
print('*********')
print(add_power.get_assignation_call_str())
print('*********')
print(add_power.get_signature())
print('*********')
print(add_power.get_parameters())
print('*********')
print(add_power.dic_imports)
print('*********')
print(add_power.set_deps)


*********
def add_power(a: int, b: int) -> int:
	"""Test with long docstring
	"""
	c = add_function(a, b)
	output_power = power_function(b, c)
	return c

*********
output_power
*********
int
*********
add_power(a, b)
*********
output_power = add_power(a, b)
*********
(a: int, b: int) -> int
*********
OrderedDict([('a', <class 'int'>), ('b', <class 'int'>)])
*********
OrderedDict([('numpy', 'np')])
*********
{'power_function', 'add_function'}
